In [1]:
!pip install openai

StatementMeta(, cb2aa45a-11ae-45ff-a2e8-0e2de8f9b6e6, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


In [2]:
from pyspark.sql import DataFrame
from typing import Iterable

StatementMeta(, cb2aa45a-11ae-45ff-a2e8-0e2de8f9b6e6, 4, Finished, Available, Finished)

In [3]:
AZURE_OPENAI_API_KEY ="7488ffc5-7e73-4071-8c83-8f6debe39d14" 
AZURE_OPENAI_ENDPOINT= "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"

StatementMeta(, cb2aa45a-11ae-45ff-a2e8-0e2de8f9b6e6, 5, Finished, Available, Finished)

In [4]:
def df_filter(df: DataFrame, conditions: Iterable[str]) -> DataFrame:
    return df.filter(" and ".join(conditions))

def get_message(content:list, question:str):
    MESSAGES = [{"role": "system", "content": "You are a helpful assistant."}]
    
    for item in content:
        tmp = {"role":"system","content":str(item)}
        MESSAGES.append(tmp)

    user_content = {"role": "user", "content": question}

    MESSAGES.append(user_content)

    return MESSAGES    

def get_content(**kwargs) -> list:
    data_df = spark.read.format("delta").load("abfss://609090c6-5f30-44b0-9851-1e1db81bd2b9@onelake.dfs.fabric.microsoft.com/58b436c1-5447-4837-a300-9856faaa0806/Tables/nyc_tlc_agg")

    conditions = []
    for key,value in kwargs.items():
        _tmp = f"{key} == '{key}: {value}'"
        conditions.append(_tmp)

    filtered_df = df_filter(data_df, conditions=conditions)

    filtered_df = filtered_df.select(filtered_df["role"],filtered_df["combined"].alias("content"))

    content_list = filtered_df.toPandas().to_dict("records")

    return content_list


def get_response(user_question: str,**kwargs):
    from openai import AzureOpenAI
    import json
        
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,  
        api_version="2024-02-01",
        azure_endpoint = AZURE_OPENAI_ENDPOINT
        )
    MODEL_NAME = 'gpt-4-turbo-2024-04-09'#'gpt-35-turbo'

    _dict = {}
    for key,val in kwargs.items():
        _dict[key] = val


    content = get_content(**_dict)

    MESSAGES = get_message(content=content,question=user_question)

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        messages=MESSAGES,
        )

    response = completion.choices[0].message

    return response.content

    


StatementMeta(, cb2aa45a-11ae-45ff-a2e8-0e2de8f9b6e6, 6, Finished, Available, Finished)

In [5]:
# def get_content(vendorID : str,input_date:str) -> list:
#     data_df = spark.read.format("delta").load("abfss://609090c6-5f30-44b0-9851-1e1db81bd2b9@onelake.dfs.fabric.microsoft.com/58b436c1-5447-4837-a300-9856faaa0806/Tables/nyc_tlc_agg")

#     if vendorID:
#         vendorID = f"vendorID: {vendorID}"
#         data_df = data_df.filter(data_df['vendorID'] == vendorID)
    
#     if input_date:
#         input_date = f"PickupDate: {input_date}"
#         data_df = data_df.filter(data_df['PickupDate'] == input_date)

#     data_df = data_df.select(data_df["role"],data_df["combined"].alias("content"))

#     content_list = data_df.toPandas().to_dict("records")

#     return content_list


# # Function with *args and **kwargs
# def example_function(**kwargs):
#     # *args: a tuple of positional arguments
#     print("Positional arguments (*args):", args)
    
#     # **kwargs: a dictionary of keyword arguments
#     print("Keyword arguments (**kwargs):", kwargs)

#     # Using *args (tuple)
#     for i, arg in enumerate(args):
#         print(f"Arg {i}: {arg}")

#     # Using **kwargs (dictionary)
#     for key, value in kwargs.items():
#         print(f"{key} = {value}")


# def sample(**kwargs):
#     _dict = kwargs.items()

#     print(_dict)

# example_function(name="Prakash",age=34,city="atlanta")

# def my_filter(df: DataFrame, conditions: Iterable[str]) -> DataFrame:
#     return df.filter(" and ".join(conditions))

# dataset = [(1,'adam',30,'atlanta','usa'),(2,'ben',32,'barcelona','spain'),(3,'charlie',35,'chicago','usa'),(4,'daniel',29,'denver','usa')]

# df = spark.createDataFrame(data=dataset, schema=['id','name','age','city','country'])

# filtered_df = my_filter(df, ["age > 30", "country == 'usa'"])

# display(filtered_df)

StatementMeta(, cb2aa45a-11ae-45ff-a2e8-0e2de8f9b6e6, 7, Finished, Available, Finished)